In [ ]:
import sys
import site

print("Python executable:", sys.executable)
print("\nPython version:", sys.version)
print("\nSite packages:", site.getsitepackages())

try:
    import numpy as np
    print("\nNumPy is available")
    print("NumPy version:", np.__version__)
    print("NumPy location:", np.__file__)
except ImportError as e:
    print("\nError importing NumPy:", str(e))
    print("Python path:", sys.path)

# Project 4: **Build a Deep Research System**
Welcome to project 4! For this project, we shift our focus from tool use and agents to *reasoning* models. You will practice state‑of‑the‑art inference‑time scaling methods such as *Chain‑of‑Thought* prompting and *Tree‑of‑Thoughts*, and briefly explore high-levels of training reasoning models using techniques like **STaR**.


Finally, you will put everything together to build a *deep research agent* that can browse the web, reason over what it finds, and give structured answers.

## Learning Objectives  
* Apply common inference‑time scaling methods: **zero‑shot / few‑shot CoT, self‑consistency, sequential decoding, tree‑of‑thoughts**  
* Gain intuition for **training** reasoning‑capable models following **STaR** approach 
* Build a minimal **deep‑research agent** that combines step‑by‑step reasoning with live web search   
* Practice extending deep-search to a multi-agent system 

## Roadmap  
1. Environment setup  
2. Inference‑time scaling  
   2.1 Few‑shot & zero‑shot CoT  
   2.2 Self‑consistency  
   2.3 Sequential revisions  
   2.4 Tree‑of‑Thought
3. STaR for training models for reasoning  
4. Deep-research agent  
5. (Optional) Multi-agent deep-research

# 1‑ Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## 1.2 Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# 2‑ Inference‑time scaling

Inference-time scaling refers to techniques that make an existing model reason better without retraining it. Instead of changing the model’s weights, we achieve reasoning capability by adjusting how we prompt, sample, or aggregate LLM's outputs.

In this section, we’ll explore several inference-time strategies that improve reasoning quality using a non-reasoning base model. You will experiment with and compare methods such as:

- Few-shot Chain-of-Thought (CoT)
- Zero-shot CoT
- Self-consistency
- Sequential revision
- Tree-of-Thoughts (ToT)

### 2.1: Few‑Shot CoT
Few-shot prompting helps a model reason by showing one or multiple examples before asking a new question. By observing the pattern of reasoning and final answers, the model learns how to structure its own reasoning process on the new input.

In this exercise, you will create a prompt that includes a few example Q&A pairs demonstrating step-by-step reasoning. Then, you will feed a new question and see the model’s output.

In [ ]:
# Step 1: Write a few examples showing reasoning steps
# Step 2: Write your new question
# Step 3: Concatenate examples + new question into a single prompt
# Step 4: Call your Ollama or OpenAI client to get a response from llama3.2:3b # e.g., client.chat.completions.create(...)
# Step 5: Print the final answer

from openai import OpenAI

client = OpenAI(api_key="20a80a4bd536439ebe175a737aa537a0.8GFpmDKt1hdC_oGr8V_KSMs6", base_url="https://ollama.com/v1")

# Run against local Ollama server
# client = OpenAI(api_key="ollama", base_url="http://ollama.")

few_shot_examples = """Q: If it is 3 PM in London (UTC+0), what time is it in New York (UTC-5)?
A: London is 5 hours ahead, so we subtract 5. The FINAL ANSWER IS: 10 AM.

Q: A tank holds 60 L of water. It leaks 3 L per hour and is filled at 5 L per hour. How much water after 4 h?
A: Net fill = 5-3 = 2 L/h. 2x4 = 8 L. The FINAL ANSWER IS: 68 L.
"""

question = "A rectangle has perimeter 40 cm and width 5 cm. What is its length?"

prompt = few_shot_examples + f"\nQ: {question}\nA:"

# LOCAL Model
MODEL_LOCAL = "llama3.2:3b"

# Cloud Model: https://docs.ollama.com/cloud#cloud-models
# deepseek-v3.1:671b-cloud
# gpt-oss:20b-cloud
# gpt-oss:120b-cloud
# kimi-k2:1t-cloud
# qwen3-coder:480b-cloud
# glm-4.6:cloud
# minimax-m2:cloud
MODEL = "deepseek-v3.1:671b-cloud"

response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)

print(response.choices[0].message.content.strip())

### 2.1.1 (Optional) Few-shot CoT on GPT2 (Local Model Run)
GPT-2 is a pre-trained language model without instruction tuning. It continues text rather than answering questions. In this section, you'll try the exact same CoT pattern on GPT-2 and observe what happens. The goal is to test whether few-shot CoT alone can elicit structured reasoning from a non-chat LLM.

#### What is PyTorch?
PyTorch is an open-source machine learning framework developed by Meta (Facebook).
It’s used for:
- Deep learning (e.g., training neural networks)
- Scientific computing (like NumPy but with GPU acceleration)

It provides tools for:
- Creating tensors (multidimensional arrays)
- Building neural networks
- Running computations efficiently on CPUs or GPUs (using CUDA, MPS, etc.)
- Automatic differentiation — PyTorch tracks operations on tensors and automatically computes gradients (used in training models)

In short: 🔹 PyTorch = NumPy + GPU acceleration + automatic differentiation + deep learning utilities


#### torch.ones()
torch.ones() creates a tensor (multi-dimensional array) filled entirely with the number 1.

It’s often used to:

- initialize weights,

- create test tensors,

- or quickly make an array of a specific shape.

```
import torch

x = torch.ones(3)
print(x)
```

Output
```
tensor([1., 1., 1.])
```

Here:

3 means the tensor has shape (3,) → a 1D tensor with 3 elements

all values are 1.0 (float by default)

##### Multi-dimensional examples

You can pass a shape tuple to make higher-dimensional tensors:
```
torch.ones(2, 3)      # 2 rows, 3 columns
torch.ones((4, 2, 2)) # 3D tensor (4 blocks of 2×2)
```

##### Optional parameters
torch.ones(size, *, dtype=None, device=None, requires_grad=False)
| Parameter       | Type                              | Description                                                                                                                                                     |
| --------------- | --------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `dtype`         | `torch.dtype`, optional           | Specifies the **data type** of the returned tensor (e.g. `torch.float32`, `torch.int64`). Default: inferred from other arguments or global default (`float32`). |
| `device`        | `torch.device` or `str`, optional | Specifies the **device** where the tensor will be allocated — typically `"cpu"`, `"cuda"`, or `"mps"`.                                                          |
| `requires_grad` | `bool`, optional                  | If `True`, PyTorch will **track gradients** for this tensor (used in automatic differentiation). Default: `False`.                                              |
| `layout`        | `torch.layout`, optional          | Specifies the **memory layout**. Usually `torch.strided` (default). Rarely changed.                                                                             |
| `pin_memory`    | `bool`, optional                  | If `True`, the tensor is created in **page-locked memory** (useful for faster CPU–GPU data transfer). Default: `False`.                                         |
| `names`         | `tuple of str`, optiona           |                                                                                                                                                                 |


#### Default device in PyTorch
If you don’t specify a device, PyTorch defaults to CPU.

In [ ]:
import os
# torch_dtype (float16 / bfloat16 / float32 / int8, etc.) influences how the model is loaded into memory.
# But in cloud mode: YOU are not loading the model
# The provider already loads the model in its own optimized configuration
# You cannot change its memory dtype
import torch
from transformers import pipeline # for local/hosted HF models

# Check the default device
# Creates a tensor (multi-dimensional array) filled entirely with the number 1.
# It’s often used to:
# - initialize weights,
# - create test tensors,
# - or quickly make an array of a specific shape.
x = torch.ones(3)
print(x.device)

# Create the tensor directly on GPU if available, otherwise on CPU
# This only matters when you load and run models locally:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

import numpy as np, torch
print(np.__version__)
print(torch.ones(1).numpy())  # should print an array([1.], dtype=float32)

#### What is a Tensor?
In mathematics, a tensor is an algebraic object that describes a multilinear relationship between sets of algebraic objects associated with a vector space

A tensor is the core data structure in PyTorch.
It’s basically a multidimensional array, like:
- 0D → scalar (a single number)
- 1D → vector
- 2D → matrix
- 3D+ → higher-dimensional arrays (for images, videos, etc.)

You can think of it as a generalization of arrays in NumPy — but tensors can also live on the GPU and participate in gradient computations.

#### What is MPS
MPS stands for Metal Performance Shaders, which is Apple’s GPU acceleration framework built on top of Metal, their low-level graphics and compute API. It’s the Apple equivalent to:
- CUDA on NVIDIA GPUs
- ROCm on AMD GPUs

MPS is used to offload tensor computations (like matrix multiplications, convolutions, etc.) to Apple’s GPU hardware, mainly on:
- Macs with Apple Silicon (M1, M2, M3 chips)
- Some Intel Macs with AMD GPUs

##### Why it matters
PyTorch uses MPS so that you can do GPU-accelerated deep learning on Mac without NVIDIA CUDA.


#### Why PyTorch doesn’t default to GPU automatically
PyTorch never automatically moves tensors to GPU, even if one is available — you must explicitly tell it to.
This design avoids unexpected memory usage or device mismatch errors.

#### Mixing devices causes errors
If you later try to perform an operation between CPU and GPU tensors:
```
x_cpu = torch.ones(3)
x_mps = torch.ones(3, device="mps")
x_cpu + x_mps   # ❌ RuntimeError
```
You’ll get an error like:
```
RuntimeError: Expected all tensors to be on the same device
```
So it’s good practice to manage your device explicitly when you plan to use GPU acceleration.

In [ ]:
# Step 1: Load GPT-2 text-generation from huggingface (https://huggingface.co/docs/transformers/en/model_doc/gpt2)
# Step 2: Write 1–2 few-shot reasoning examples (short, explicit steps + final answer in your own unique format)
# Step 3: Append a new test question after the examples to form one prompt string
# Step 4: Generate 1–3 completions with different decoding settings (e.g., greedy vs. top-k)
# Step 5: Print raw outputs; check if steps are followed and if the final answer is correct

# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# assert torch.backends.mps.is_available(), "MPS not available on this system"
# torch.mps.empty_cache()
# device = torch.device("mps")

pipeline = pipeline("text-generation", 
                   model="openai-community/gpt2", 
                   torch_dtype=torch.float32,  # Changed from float16 to float32
                   device=device)

In [ ]:
# import sys
# print("Python executable:", sys.executable)
# print("\nPython version:", sys.version)

# import numpy as np
# print("\nNumPy version:", np.__version__)

In [ ]:
pipeline

In [ ]:
few_shot_examples = """Q: If it is 3 PM in London (UTC+0), what time is it in New York (UTC-5)?
A: London is 5 hours ahead, so we subtract 5. The FINAL ANSWER IS: 10 AM.

Q: A tank holds 60 L of water. It leaks 3 L per hour and is filled at 5 L per hour. How much water after 4 h?
A: Net fill = 5-3 = 2 L/h. 2x4 = 8 L. The FINAL ANSWER IS: 68 L.
"""

question = "A rectangle has perimeter 40 cm and width 5 cm. What is its length?"

prompt = few_shot_examples + f"\nQ: {question}\nA:"


In [ ]:
greedy_out = pipeline(prompt, max_new_tokens=128, do_sample=False, use_cache=False)
print(greedy_out[0]["generated_text"])

In [ ]:
topp_out = pipeline(prompt, max_new_tokens=128, do_sample=True, top_p=0.95, use_cache=False)
print(topp_out[0]["generated_text"])

In [ ]:
print(x.device)

### 2.2: Zero‑Shot Chain‑of‑Thought
Zero-shot CoT encourages the model to reason without examples by adding a short cue such as “Let’s think step by step.” This simple phrase often activates the model’s latent reasoning ability even when no demonstrations are provided. It serves as a baseline to compare with few-shot and other inference-time scaling methods.

In [ ]:
from openai import OpenAI

# Step 1: Write the question and a zero-shot CoT cue (e.g., "Let's think step by step.")
# Step 2: Build a single prompt string that includes brief role guidance plus the question
# Step 3: Call your Ollama or OpenAI client to get a response from llama3.2:3b  # e.g., client.chat.completions.create(...)
# Step 4: Print the chain and the final answer

client = OpenAI(api_key="20a80a4bd536439ebe175a737aa537a0.8GFpmDKt1hdC_oGr8V_KSMs6", base_url="https://ollama.com/v1")

# client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
question = "Why do we use neural networks to build LLMs?"

prompt = f"You are a knowledgeable tutor. Answer the quetion. Question: {question} \n Let's think step by step."

CLOUD_MODEL = "deepseek-v3.1:671b-cloud"
LOCAL_MODEL = "llama3.2:1b"

response = client.chat.completions.create(
    model=CLOUD_MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.1,
)


In [ ]:
print(response.choices[0].message.content.strip())

### 2.3 Self‑Consistency
Self-consistency enhances reasoning accuracy by sampling multiple independent reasoning paths for the same question instead of relying on a single deterministic answer. Each run may follow a slightly different logical chain, and the diversity helps correct individual mistakes. After generating several reasoning traces, you then aggregate the final answers using majority voting.

This approach is especially useful when tasks involve multi-step reasoning or arithmetic, where single-path outputs may be incorrect.

In [ ]:
from openai import OpenAI
import re, collections

client = OpenAI(api_key="20a80a4bd536439ebe175a737aa537a0.8GFpmDKt1hdC_oGr8V_KSMs6", base_url="https://ollama.com/v1")
# client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")
MODEL = "llama3.2:1b"
CLOUD_MODEL = "deepseek-v3.1:671b-cloud"

def cot_answer(question, temperature=0.1):
    # Generate a step-by-step reasoning chain for the given question and extract the final answer.
    prompt = f"""Answer the following question with step-by-step reasoning and final answer after
        **Therefore, (answer here)**.
        Question: {question}
        Let's think step by step.
    """
    response = client.chat.completions.create(
        model=CLOUD_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    content = response.choices[0].message.content
    match = re.search(r"[Tt]herefore,?\s*(.*)", content) # Extract text after "Therefore,"
    return content, match.group(1).strip() if match else None

question = "What is the square root of 144?"
print(cot_answer(question))
# winner, counter = self_consistent(question)
# print("Votes:", counter)
# print("Chosen answer:", winner)

In [ ]:
def self_consistent(question, n=10):
    # Run multiple reasoning chains and select the most frequent final answer by majority voting.
    answers = []
    for _ in range(n):
        _, ans = cot_answer(question)
        answers.append(ans)
    counter = collections.Counter(answers)
    winner, _ = counter.most_common(1)[0]
    return winner, counter


question = "What is the square root of 144?"
winner, counter = self_consistent(question)
print("Votes:", counter)
print("Chosen answer:", winner)

### 2.4: Sequential Revision

Sequential revision iteratively improves an answer by generating a first draft, critiquing it, and producing revised drafts that condition on prior answers. Each round should be short and focused, so improvements accumulate without drifting from the question.

In [ ]:
CLOUD_MODEL = "deepseek-v3.1:671b-cloud"
MODEL = "llama3.2:3b"

def sequential_revision(question: str, max_steps: int = 3) -> str:
    # Generate an initial draft answer, then iteratively refine it by conditioning each revision on the previous one.
    # Step 1: Ask the model to produce the first draft for the given question
    # Step 2: Loop for max_steps-1 times, each time feeding the last draft back to the model with a request to revise
    # Step 3: Print each draft to observe how the answer evolves
    # Step 4: Return the final improved draft
    message = [
        {"role": "system", "content": "You are a helpful assistant. Keep your answers clear and correct"},
        {"role": "user", "content": question}
    ]

    draft = client.chat.completions.create(
        model=CLOUD_MODEL,
        messages = message,
        temperature=0.1
    ).choices[0].message.content.strip()

    for idx in range(max_steps):
        messages = [
             {"role": "system", "content": "You are a helpful assistant. Improve answers by making them clearer and more accurate."},
             {"role": "user", "content": question},
             {"role": "assistant", "content": draft},
             {"role": "user", "content": "Please revise your answer. Make it clearer, more accurate, and better written. Only include the new answer."}
        ]
        draft = client.chat.completions.create(
            model=CLOUD_MODEL,
            messages=messages,
            temperature=0.1
        ).choices[0].message.content.strip()
        print(f"Draft {idx+1}: {draft}")
    
    return draft

# Step 1: Define a question that benefits from multi-step reasoning
# Step 2: Call sequential_revision(question, max_steps)
# Step 3: Print the final output
output = sequential_revision("If a rectangle is twice as long as it is wide and the perimeter is 30cm, what is the area?")
output

### 2.5 Tree‑of‑Thoughts
Tree-of-Thoughts reframes reasoning as a search process rather than a single forward chain.
Instead of producing one linear sequence of thoughts, the model generates multiple candidate thoughts at each step, evaluates their promise, and then expands only the best few. This allows exploration of different reasoning paths before committing to a final answer, similar to how humans brainstorm, prune, and refine ideas.


In this section, you’ll experiment with two simplified versions of ToT:
1. Word Ladder puzzle solver: a small example where each “thought” is a candidate word transition.
2. Generic ToT search (depth 2, width 2): a minimal logic to expand, evaluate, and select reasoning branches

In [ ]:
###### Word Ladder Puzzle ##########

def neighbors(word, vocabulary):
    # Generate all valid one-letter mutations of 'word' that exist in 'vocabulary' and return them.
    for i, c1 in enumerate(word):
        for c2 in "abcdefghijklmnopqrstuvwxyz":
            if c1 != c2:
                candidate = word[:i] + c2 + word[i+1:]
                if candidate in vocabulary:
                    yield candidate


def tree_of_thought(start, goal, vocab, max_depth=5, beam_width=4):
    # Search over partial thoughts (paths) using a small beam.
    # Step 1: Initialize the frontier with a single path [start]
    # Step 2: For each depth, expand each path by one neighbor from 'neighbors'
    # Step 3: Score paths by edit distance between last word and 'goal' (smaller is better)
    # Step 4: Keep the top 'beam_width' paths and stop early if any reaches 'goal'
    # Step 5: Return the best goal-reaching path or None
    frontier = [[start]]
    for depth in range(max_depth):
        candidates = []
        for path in frontier:
            for nxt in neighbors(path[-1], vocab):
                if nxt in path:  # avoid loops
                    continue
                candidates.append(path + [nxt])
        # score: negative edit distance to goal
        scored = sorted(candidates, key=lambda p: sum(a!=b for a,b in zip(p[-1], goal)))
        frontier = scored[:beam_width]
        if any(p[-1] == goal for p in frontier):
            return [p for p in frontier if p[-1]==goal][0]
    return None


vocab = {"hit","dot","cog","log","dog","lot","lit","hot"}
print(tree_of_thought("hit", "cog", vocab)) # one candidate solution: ['hit', 'hot', 'dot', 'dog', 'cog']


In [ ]:
###### Generic ToT Search ##########

import re

MODEL = "llama3.2:3b"
# client = OpenAI(api_key="20a80a4bd536439ebe175a737aa537a0.8GFpmDKt1hdC_oGr8V_KSMs6", base_url="https://ollama.com/v1")

def propose_thoughts(question, state, k=2):
    # Propose up to k next “thoughts” that extend the current partial solution/state.
    # Steps: build a short prompt with problem + current state; call your client with n=k. Then return a list of stripped strings (≤ k).
    prompt = f"""You are exploring solutions.
            Problem: {question}
            Current partial solution: {state}
            
            Propose at most {k} different next thoughts."""
    r = client.chat.completions.create(
        model=CLOUD_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        n=k
    )
    return [c.message.content.strip() for c in r.choices]

def score_state(question, state):
    # Score how promising a partial solution is on a 1–10 scale (higher is better).
    # Steps: build a rating prompt; call the model; parse the first integer 1–10;
    prompt = f"""Problem: {question}
        Rate from 1-10 how promising this partial solution is: {state}"""
    
    r = client.chat.completions.create(
        model=CLOUD_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    try:
        return int(re.findall(r"\d+", r.choices[0].message.content)[0])
    except Exception:
        return 5 # neutral fallback


def tree_of_thoughts(question, depth=2, width=2):
    # Run a tiny ToT search: expand states with propose_thoughts, score with score_state, keep top-k at each depth.
    # Steps: initialize frontier=[("", 0)]; for each depth, expand each state with k=width thoughts; score each; sort by score desc; keep top 'width'; return best state and score.
    frontier = [("", 0)]
    for _ in range(depth):
        new_frontier = []
        for state, _ in frontier:
            for thought in propose_thoughts(question, state, k=width):
                new_state = (state + "\n" + thought).strip()
                score = score_state(question, new_state)
                new_frontier.append((new_state, score))
        new_frontier.sort(key=lambda x: x[1], reverse=True)  # keep top‑k
        frontier = new_frontier[:width]
    best_state, best_score = frontier[0]
    return best_state, best_score


question = "Design a plan for a weekend science workshop for 12-year-olds."
solution, score = tree_of_thoughts(question)

print(f"Best solution (score {score}):\n{solution}")

---  
# 3‑ Training Models for Reasoning

### 3.1: CoT Training
Chain-of-Thought (CoT) training conditions the model on explicit rationales during fine-tuning. Instead of teaching the model to output only the final answer, we train on (question, rationale, answer) so the model learns to internalize multi-step reasoning patterns. A practical recipe is STaR (Self-Taught Reasoner), which uses a stronger teacher model to bootstrap rationales that a smaller student can learn from.

For tasks that require multi-hop reasoning, models fine-tuned on rationales often achieve higher accuracy and are more stable at inference time than models trained on direct answers only. 

Training a full language model is beyond the scope of this notebook, but here is the high-level workflow followed by a short pseudocode:
- Collect questions: Prepare a dataset of questions and correct answers.
- Generate rationales: Use a strong LLM to produce step-by-step reasoning ending with the correct answer.
- Filter and clean: Discard incorrect or low-quality rationales.
- Prepare training data: Format triples (question, rationale, answer) for supervised fine-tuning.
- Fine-tune: Fine-tune the LLM on rationales.
- Iterate: Refine prompts, improve data quality, and retrain for stronger reasoning.

In [ ]:
# Pseudocode (STaR loop)
# for round in 1 ... iters:
    # STEP 1: self-generate reasoning (teacher creates rationale + answer)
    # STEP 2: keep only correct, high-quality traces
    # STEP 3: fine-tune student on (question, rationale, answer) data

### 3.2: ORM vs PRM + RL
Training a Reward Model (RM) allows large language models to be improved through reinforcement learning (RL). Instead of fine-tuning directly on examples, we train a separate model that can score or rank model outputs, and use those scores as feedback signals to refine the policy model.

Two main reward modeling approaches are ORM (predicts a scalar reward for the final answer) and PRM (evaluates the reasoning steps instead of just the outcome)



| Approach | Typical loss | When to use |
|-----------|-------------|-------------|
|*Outcome Reward Model* | Predict scalar reward | Easy to collect training data using verifiers |
|*Process Reward Model* | Predict rewards per step | Difficult to collect training data but more accurate |
| *RLHF* | Use RM as reward in **RL** fine‑tuning | Aligns policy with human signals | Aligns model policy with human or synthetic preferences




In [ ]:
# for round = 1 ... iters:
    # STEP 1:  Generate reasoning
        # sample a minibatch of questions
        # policy roll‑out (actions + log‑probs)
    # STEP 2:  Score the trajectory
        # ORM: scalar reward for the final answer / PRM: scalar reward for the thought process
    # STEP 3:  Reinforce the policy (PPO)

---  
# 4‑ A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [ ]:
from ddgs import DDGS
from langchain.tools import Tool

def ddg_search(query: str, k: int = 5) -> str:
    # Use DDGS to run a simple web search and return joined snippets.
    with DDGS() as ddgs:
        results = [hit["body"] for hit in ddgs.text(query, max_results=k)]
    return "\n".join(results)

search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

MODEL = "deepseek-r1:8b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
llm = ChatOllama(model=MODEL, temperature=0.2)

# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)


# Step 3: Ask a query and let the agent search + reason to produce an answer
result = agent.invoke({"input": question})
print(result["output"])

# Optional (Multi-agent Deep Research)
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Try building a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [ ]:
def parallel_research(query, n=3):
    # Run n independent research runs in parallel and return their answers.
    # Steps: use ThreadPoolExecutor; submit n calls to your agent/search pipeline; gather results in order.
    """
    YOUR CODE HERE
    """

answers = parallel_research("What are the best resources to learn ML in 2025?")
for i,a in enumerate(answers,1):
    print(f"[Run {i}] {a[:200]}…")

## 🎉 Congratulations!

* Practised various inference‑time reasoning methods
* Gained intuition about training reasoning models
* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* Try adding more tools, and extending the deep-research to a multi-agent system: many agents researching web in parallel.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.